# 3.9.1 同性婚に関する意見の変化再考

## 問題1
- 研究1のコントロールグループについて、第1波と第2波の感情温度の相関をみてみなはれ
    - 欠損のない観察のみを用いてやってね
- 結果について簡潔に実質的な解釈を述べなはれ

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)

# 研究1のコントロールグループについて、第1波と第2波の感情温度の相関をみてみなはれ
# 欠損のない観察のみを用いてやってね
import scipy.stats
gayreshaped = pd.read_csv("officialSource/qss-master/MEASUREMENT/gayreshaped.csv")
study1 = gayreshaped[gayreshaped["study"]==1]
study1_noNull = study1[(~study1[["therm1", "therm2"]].isnull().any(axis=1)) & (study1["treatment"]=="No Contact")]
# 相関係数出してみる
scipy.stats.pearsonr(study1_noNull["therm1"], study1_noNull["therm2"])
# ちなみに、NaNがある状態で相関係数だそうとしても、scipy.stats.pearsonrだとnanになる
scipy.stats.pearsonr(study1["therm1"], study1["therm2"])

相関係数＝0.9975816546030554, p値＝0.0と、相関しているというかほぼ同一なのでは？という疑惑。

In [ ]:
study2 = gayreshaped[gayreshaped["study"]==2]
study2_controll = study2[study2["treatment"]=="No Contact"]
study2_controll.corr()

問題1と同様、いずれの波でも高い相関係数となっている。

## 問題3
- 外れ値の確認、および、第1波とそれぞれの波の変化の様子がわかる散布図を描いてみなはれ

In [ ]:
# 2×2に sns.jointplotを並べようとしたけど、うまく描画できなかったので諦める
# fig = plt.figure(figsize=(16,12))
# あ、こういう書き方できるんだ！！
# [[ax1, ax2], [ax3, ax4]] = fig.subplots(nrows=2, ncols=2)

# jointplotで戻ってくるのは、JointGridオブジェクトが返ってくる。
# xxxGrid 系は、seaborn内部で複数のaxesをFigureオブジェクト内に配置してくれるやつで、
# JointGridも、X軸とY軸の上に出ている分布図も、そういう大きさのaxesを配置してくれている。
# なので、JointGrid全体のタイトルをつける場合、全体のタイトルに見えやすい位置に配置されるタイトルとして
# jointgrid.ax_marg_x のタイトルに名前を付ける
jointgrid = sns.jointplot(data=study2_controll, x="therm1", y="therm2", kind="reg")
jointgrid.ax_joint.axes.title.set_text("")
jointgrid.ax_marg_x.title.set_text("第1波と第2波の分布比較")
jointgrid.fig.show()

jointgrid2 = sns.jointplot(data=study2_controll, x="therm1", y="therm3", kind="reg")
jointgrid2.ax_marg_x.title.set_text("第1波と第3波の分布比較")
jointgrid2.fig.show()

jointgrid3 = sns.jointplot(data=study2_controll, x="therm1", y="therm4", kind="reg")
jointgrid3.ax_marg_x.title.set_text("第1波と第4波の分布比較")
jointgrid3.fig.show()

## 問題4
- CCAPのデータセットと酷似している事を見出した研究者3人の足跡を追ってみよう
    - 2012年CCAPの感情温度、研究1第1波の感情温度、研究2の第1波の感情温度のヒストグラムを見てみる
    - CCAPには多くの欠損値が含まれている。同性婚研究において、欠損データがどのように記録されたか検討しよう

In [ ]:
ccap2012 = pd.read_csv("officialSource/qss-master/MEASUREMENT/ccap2012.csv")
figure = plt.figure(figsize=(24,6))
ax1, ax2, ax3 = figure.subplots(nrows=1, ncols=3)
# 研究1第1波のヒストグラム
ax1 = sns.distplot(study1["therm1"], ax=ax1, bins=20, norm_hist=True)
ax1.set_ylim(0, 0.07)
ax2 = sns.distplot(study2["therm1"], ax=ax2, bins=20, norm_hist=True)
ax2.set_ylim(0, 0.07)
ax3 = sns.distplot(ccap2012[~ccap2012["gaytherm"].isnull()]["gaytherm"], ax=ax3, bins=20, norm_hist=True)
ax3.set_ylim(0, 0.07)
figure.show()

# CCAPのデータ分布と、今回のデータ、真ん中の分布が多いだけで、後はほとんど一緒だな・・・
# ど真ん中、つまりtherm値が50の人を、CCAPのデータから水増ししてんじゃね・・疑惑

# therm値50の人の割合を見てみると、そこだけで2割弱。飛び抜けて多いのは怪しい・・・
(study1["therm1"]==50).mean()
(study2["therm1"]==50).mean()

# CCAP の欠損データを、50で埋めたら似た分布になったりしてね
ccap2012_fillna_with50 = ccap2012.copy(deep=True)
ccap2012_fillna_with50.loc[ccap2012["gaytherm"].isnull(), "gaytherm"] = 50
ax4 = sns.distplot(ccap2012_fillna_with50["gaytherm"], bins=20, norm_hist=True)
ax4.figure.show()
# はいそっくりになったーーーー。
# なので、CCAPの欠損データをすべて50で埋めて作られていると思われる

## 問題5
- 問題4で扱ったデータを、QQプロット(的なん)で可視化してみなはれ。多分キレイに一致してるから。

In [ ]:
# 累積分布を、ｘ軸をポジション化(分位値化)してやるやーつ
import probscale
position1, values1 = probscale.plot_pos(study1["therm1"])
position2, values2 = probscale.plot_pos(study2["therm1"])
position3, values3 = probscale.plot_pos(ccap2012_fillna_with50["gaytherm"])

temp1 = pd.DataFrame({"x":position1, "y":values1, "hue":["study1" for i in range(1, len(position1) + 1)]})
temp2 = pd.DataFrame({"x":position2, "y":values2, "hue":["study2" for i in range(1, len(position2) + 1)]})
temp3 = pd.DataFrame({"x":position3, "y":values3, "hue":["ccap" for i in range(1, len(position3) + 1)]})

ax = sns.scatterplot(x="x", y="y", style="hue", hue="hue", data=temp1.append(temp3).reset_index(drop=True))
ax.figure.show()

ax = sns.scatterplot(x="x", y="y", hue="hue", data=temp2.append(temp3).reset_index(drop=True))
ax.figure.show()

# ほとんど丸かぶりで、データ生成法としての仮説は正しいだろう。

# 3.9.2 中国とメキシコにおける政治的有効性感覚

## 問題1
- 中国とメキシコの自己評価回答の分布を別々に棒グラフにせよ
    - 縦軸には回答者の割合
- それぞれお国の回答の平均を計算しまひょ
_ 上記結果から、どちらの国のほうが世辞的有効性感覚が高いといえるか？
    - メキシコは、70年以上統治してきた制度的革命党を2000年の選挙で政権から追い出し、他方中国は現在まで公正な選挙が行われたことはない。
    - この事実は、分析結果と合致するか？

In [ ]:
vignettes = pd.read_csv("officialSource/qss-master/MEASUREMENT/vignettes.csv")
myfig = plt.figure(figsize=(16,6))
ax1, ax2 = myfig.subplots(nrows=1, ncols=2)
ax1 = sns.distplot(vignettes[vignettes["china"]==0]["self"], kde=False, ax=ax1, bins=[0,1,2,3,4,5,6], norm_hist=True)
ax2 = sns.distplot(vignettes[vignettes["china"]==1]["self"], kde=False, ax=ax2, bins=[0,1,2,3,4,5,6], norm_hist=True)
ax1.set_ylim(0, 0.6)
ax2.set_ylim(0, 0.6)
ax1.title.set_text("メキシコの自己評価分布")
ax2.title.set_text("中国の自己評価分布")
myfig.show()

# それぞれの平均値
# メキシコ
vignettes[vignettes["china"]==0]["self"].mean()
# 中国
vignettes[vignettes["china"]==1]["self"].mean()


分布的にはメキシコの方が政治的有効性の自己評価は低い側に偏っており、中国は比較的平ら。
平均値も、メキシコが 1.8253、中国が 2.6219 と中国の方が高い。
政治的背景を考えると、メキシコの方が高いと予想していたが、予想とは違う結果となっている。

## 問題2
両国の政治的有効性感覚の程度の違いが、年齢分布に起因するかどうかを検討する。
- メキシコと中国の回答者の年齢ヒストグラムを作成しましょ
    - 回答者の年齢の中央値を示す縦線を描いておきましょ
- 2国間の年齢分布比較するためにQQプロット描いてみましょ
- それぞれのグラフを解釈しながら、考察してみな

In [ ]:
# 各国の年齢分布を描いてみる
figure = plt.figure(figsize=(16, 6))
ax1, ax2 = figure.subplots(nrows=1, ncols=2)
sns.distplot(vignettes[vignettes["china"]==0]["age"], 
             ax=ax1, kde=False, norm_hist=True,
             bins=[0,10,20,30,40,50,60,70,80,90,100]
             )
sns.distplot(vignettes[vignettes["china"]==1]["age"],
             ax=ax2, kde=False, norm_hist=True,
             bins=[0,10,20,30,40,50,60,70,80,90,100]
             )
ax1.title.set_text("メキシコの年齢分布")
ax2.title.set_text("中国の年齢分布")
ax1.set_ylim(0,0.05)
ax2.set_ylim(0,0.05)
# 中央値の垂直線を追加
ax1.axvline(x=vignettes[vignettes["china"]==0]["age"].median())
ax1.text(x=vignettes[vignettes["china"]==0]["age"].median(), y=0.03, s="中央値", ha="left", va="center")
ax2.axvline(x=vignettes[vignettes["china"]==1]["age"].median())
ax2.text(x=vignettes[vignettes["china"]==1]["age"].median(), y=0.03, s="中央値", ha="left", va="center")
figure.show()
# ちなみに中央値は・・メキシコが35歳、中国が45歳
vignettes[vignettes["china"]==0]["age"].median()
vignettes[vignettes["china"]==1]["age"].median()

# QQプロット的なものを描く
import probscale
mexico_pos, mexico_val = probscale.plot_pos(vignettes[vignettes["china"]==0]["age"])
china_pos, china_val = probscale.plot_pos(vignettes[vignettes["china"]==1]["age"])

temp1 = pd.DataFrame(data={"x":mexico_pos, "y":mexico_val, "country":["mexico" for i in range(1, len(mexico_pos)+1 )]})
temp2 = pd.DataFrame(data={"x":china_pos, "y":china_val, "country":["china" for i in range(1, len(china_val)+1 )]})
temp = temp1.append(temp2).reset_index(drop=True)

fig = plt.figure()
ax = fig.subplots()
sns.scatterplot(ax=ax, x="x", y="y", hue="country", data=temp, alpha=0.3)
ax.figure.show()